# load weather data (US version)

In [184]:
import pandas as pd
from datetime import datetime,timedelta
import re
import numpy as np

In [215]:
#set date range
d1 = datetime(2010,1,1)
d2 = datetime(2017,7,18)

delta = d2-d1

In [216]:
span = []
for i in range(delta.days+1):
     intdate = map(int, str(d1+timedelta(days=i)).split(' ')[0].split('-'))
     span.append(intdate)

In [217]:
allhours=[]
for i in range(int(delta.total_seconds()//3600)+1):
     intdate = (d1+timedelta(hours=i))
     allhours.append(intdate)

In [218]:
len(span)

41

In [219]:
#loop the dates
for ind, day in enumerate(span):
    print day
    
    #load data from url
    y,m,d = (day[0],day[1],day[2])
    url = 'https://www.wunderground.com/history/airport/KBWI/{}/{}/{}/DailyHistory.html?req_city=Baltimore&req_state=MD&req_statename=&reqdb.zip=21240&reqdb.magic=7&reqdb.wmo=99999'.format(y,m,d)
    w = pd.read_html(url)[4]
    
    #select target columns and unify names
    cols = w.columns
    w.rename(columns={cols[0]:'time'},inplace=True)
    w = w[['time','Temp.','Dew Point', 'Humidity', 'Pressure','Visibility','Wind Dir','Wind Speed','Gust Speed', 'Precip','Events','Conditions']]
    cols = w.columns
    
    #reform time format
    w['datetime'] = [('-').join(map(str,(y,m,d)))+' '+'00'+':'+t.split(':')[1].split(' ')[0] if('12'in t.split(':')[0])and ('AM'in t)
             else ('-').join(map(str,(y,m,d)))+' '+t.split(' ')[0] if 'AM' in t 
             else ('-').join(map(str,(y,m,d)))+' '+'12'+':'+t.split(':')[1].split(' ')[0] if('12'in t.split(':')[0])and ('PM'in t)
             else ('-').join(map(str,(y,m,d)))+' '+str(int(t.split(':')[0])+12)+':'+t.split(':')[1].split(' ')[0]
             for t in w[cols[0]]]
    
    #remove the units 
    w.fillna(value='-',inplace=True)
    w[cols[7]]=[v.split(' ')[0] for v in w[cols[7]]]
    w[cols[8]]=[v.split(' ')[0] for v in w[cols[8]]]
    
    for i in [1,2,3,4,5,7,8,9]:
        w[cols[i]]=[re.sub('[^-0-9.]','',v) for v in w[cols[i]]]
        
    #reset column names
    w.columns = ['time','temp.(c)', 'dew_point(c)','humidity(%)', 'pressure(hpa)','visibiliy(km)', 
             'wind_dir', 'wind_speed(km/h)','gust_speed(km/h)','precip(mm)','events','conditon','datetime']
    
    #set hourly time
    w['datetime'] = pd.to_datetime(w['datetime'])
    w['datetime_hourly'] = [t+timedelta(minutes=(60-t.minute)) if (60-t.minute)<30 
                            else t-timedelta(minutes=t.minute) for t in w['datetime']]
    w['to_whole_point(min)'] = [(60-t.minute) if (60-t.minute)<30 
                            else t.minute for t in w['datetime']]
    
    #remove the duplicates
    w.sort_values(['datetime_hourly','to_whole_point(min)'],inplace=True)
    w.drop_duplicates(subset='datetime_hourly',keep='first',inplace=True)
    
    #save file
    with open ('../weather_bwi_t.csv','a') as f:
        #if ind ==0:
            #w.to_csv(f, index= False, header = True,encoding = 'utf-8')
        #else:
        w.to_csv(f, index= False, header = None,encoding = 'utf-8')

#total = pd.concat(frames)
#total['location'] = 

#total.to_csv('../total_weather_us.csv', index= False, header = True, encoding = 'utf-8')


[2017, 6, 8]
[2017, 6, 9]
[2017, 6, 10]
[2017, 6, 11]
[2017, 6, 12]
[2017, 6, 13]
[2017, 6, 14]
[2017, 6, 15]
[2017, 6, 16]
[2017, 6, 17]
[2017, 6, 18]
[2017, 6, 19]
[2017, 6, 20]
[2017, 6, 21]
[2017, 6, 22]
[2017, 6, 23]
[2017, 6, 24]
[2017, 6, 25]
[2017, 6, 26]
[2017, 6, 27]
[2017, 6, 28]
[2017, 6, 29]
[2017, 6, 30]
[2017, 7, 1]
[2017, 7, 2]
[2017, 7, 3]
[2017, 7, 4]
[2017, 7, 5]
[2017, 7, 6]
[2017, 7, 7]
[2017, 7, 8]
[2017, 7, 9]
[2017, 7, 10]
[2017, 7, 11]
[2017, 7, 12]
[2017, 7, 13]
[2017, 7, 14]
[2017, 7, 15]
[2017, 7, 16]
[2017, 7, 17]
[2017, 7, 18]


In [221]:
#recheck the cross-date duplicates
d = pd.read_csv('../weather_bwi_t.csv',header=0)
print d.head()
d.drop_duplicates(subset='datetime_hourly',keep='first',inplace=True)
d.columns = ['time','temp.(c)', 'dew_point(c)','humidity(%)', 'pressure(hpa)','visibiliy(km)', 
             'wind_dir', 'wind_speed(km/h)','gust_speed(km/h)','precip(mm)','events',
             'conditon','datetime','datetime_hourly','to_whole_point(min)']
d['temp.(c)']=[np.nan if t == '-' else t for t in d['temp.(c)']]
d['temp.(c)'].astype(float)
d.to_csv('../weather_bwi_t.csv',header=True, index=False)

       time  temp.(c)  dew_point(c)  humidity(%)  pressure(hpa)  \
0  12:54 AM       6.1          13.3         57.0         1020.6   
1   1:54 AM       6.7          12.8         62.0         1021.0   
2   2:54 AM       8.9          12.8         74.0         1021.0   
3   3:54 AM       8.3          12.8         71.0         1020.9   
4   4:54 AM       7.8          12.8         68.0         1021.0   

   visibiliy(km) wind_dir  wind_speed(km/h)  gust_speed(km/h)  precip(mm)  \
0           16.1       NW               7.4               NaN         NaN   
1           16.1      WNW              11.1               NaN         NaN   
2           16.1     West               9.3               NaN         NaN   
3           16.1      WNW              11.1               NaN         NaN   
4           16.1      WNW              14.8               NaN         NaN   

  events       conditon             datetime      datetime_hourly  \
0      -  Partly Cloudy  2002-01-01 00:54:00  2002-01-01 01:00:00